## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [ ]:
use_hydroelastic = True # this takes forever

def make_builder():

    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    return builder, plant, station

def add_controller(builder, station, traj = None, wsg_src = None):

    # ID CONTROLLER
    _, iiwa_plant, _ = build_temp_plant()
    ID_controller = builder.AddSystem(
        InverseDynamicsController(
            iiwa_plant,
            kp = np.ones(7) * 200,
            kd = np.ones(7) * 40,
            ki = np.zeros(7),
            has_reference_acceleration=False,
        )
    )


    control_estimated_state_input_port = ID_controller.get_input_port(0)
    control_desired_state_input_port = ID_controller.get_input_port(1)
    control_output_port = ID_controller.get_output_port(0)


    state_src = builder.AddSystem(TrajectorySource(traj, output_derivative_order=1))

    builder.Connect(
        state_src.get_output_port(),
        control_desired_state_input_port
    )

    builder.Connect(
        station.GetOutputPort("iiwa_state"),
        control_estimated_state_input_port
    )

    builder.Connect(
        control_output_port,
        station.GetInputPort("iiwa_actuation")
    )

    wsg_src = builder.AddSystem(TrajectorySource(wsg_traj))
    builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))
    return None

# diagram = builder.Build()

In [5]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

# builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WPregrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.7, 0.5])
X_WGrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.5])
X_WLift = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.7])
AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrasp)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrasp)

for pos in [X_WPregrasp, X_WGrasp, X_WLift]:
    print(solve_ik_for_pose(plant, pos))
# AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
# AddMeshcatTriad(meshcat, 'b', X_PT=X_WGoal)


traj, wsg_traj = grasp_path(X_WStart, X_WPregrasp, X_WGrasp, X_WLift)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

NameError: name 'X_WBroom' is not defined

In [7]:
from scripts.broom_utils import get_broom_pregrip, get_broom_grip,compute_broom_grasp_angle
# from scripts.meta_controller import pregrip_to_grip_trajectory

plant_context = plant.CreateDefaultContext()
base = plant.GetBodyByName("base", plant.GetModelInstanceByName('iiwa'))
body = plant.GetBodyByName("handle_link")
X_WBroom = plant.EvalBodyPoseInWorld(plant_context, body)
X_WBase = plant.EvalBodyPoseInWorld(plant_context, base)
ang = compute_broom_grasp_angle(X_WBroom, X_WBase.translation())
print(ang)
X_WPregrip = get_broom_pregrip(X_WBroom, ang)
# X_WPregrip = X_WGoal
X_WGrip = get_broom_grip(X_WBroom, ang)
# AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
# AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
# AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrip)
import random
for i in range(1):
    x = random.uniform(-1, 1)
    y = random.uniform(1.6, 2)
    trans = RigidTransform(RotationMatrix(), [x, y, 0])
    ang = compute_broom_grasp_angle(trans, X_WBase.translation())
    grip = get_broom_grip(trans, ang)
    AddMeshcatTriad(meshcat, f'{i}', X_PT=grip)
AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrip)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrip)


NameError: name 'plant' is not defined

In [61]:


traj_V_G, traj_wsg = plan_path(X_WStart, X_WPregrip)
builder, plant, station = make_builder(traj_V_G, traj_wsg)
traj = traj_V_G
# traj2_V_G, traj2_wsg_command = pregrip_to_grip_trajectory(X_WPregrip, X_WGrip)

# traj_V_G = combine_trajectory(traj1_V_G, traj2_V_G)
# traj_wsg = combine_trajectory(traj1_wsg, traj2_wsg)

# grip the broom

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
# q_src = builder.AddSystem(TrajectorySource(traj))
# builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
# wsg_src = builder.AddSystem(TrajectorySource(traj_wsg))
# builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
X_WPregrip


7
(7, 10)
Collision optimization failed


time0.0: [ 0.63340242 -1.14653133 -1.26941602 -1.37945514 -0.61520393  1.72992841
 -0.24633868]
time0.06881152756854825: [ 0.6277118  -1.11639413 -1.23060293 -1.37664382 -0.61666536  1.70813181
 -0.29178414]
time0.1376230551370965: [ 0.61332747 -1.04263926 -1.13589901 -1.36975491 -0.62021083  1.65506979
 -0.4030179 ]
time0.20643458270564474: [ 0.59426765 -0.95015685 -1.01778049 -1.36109747 -0.62458714  1.58916047
 -0.54252992]
time0.275246110274193: [ 0.57341319 -0.85556182 -0.89768172 -1.35223784 -0.62897329  1.52241182
 -0.68534302]
time0.34405763784274124: [ 0.55164106 -0.76088957 -0.77758295 -1.34343371 -0.63330977  1.45553815
 -0.82856095]
time0.4128691654112895: [ 0.52960429 -0.66665081 -0.65748418 -1.33480189 -0.63761787  1.38812273
 -0.97158911]
time0.48168069297983773: [ 0.5076203  -0.57267428 -0.53738541 -1.3263301  -0.64197382  1.32006016
 -1.11474001]
time0.550492220548386: [ 0.4857281  -0.47822285 -0.41728663 -1.31789891 -0.64649945  1.25150337
 -1.25907882]
time0.61930374

RigidTransform(
  R=RotationMatrix([
    [0.7071069061514456, 0.6123718945812437, 0.35355407790026816],
    [-0.7071066562216274, 0.6123721110266624, 0.35355420286544226],
    [0.0, -0.5000010603626028, 0.866024791582939],
  ]),
  p=[0.4897730589753761, 1.9897732320877213, 0.6132051491818563],
)

In [62]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

: 